In [1]:
import os

import keras
from keras.layers import (Activation, BatchNormalization, Conv2D, Dense,
                          Dropout, Flatten, MaxPooling2D)
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
num_classes = 7
img_rows,img_cols = 48 , 48
batch_size = 2

train_data_dir = r"E:\ML\FACE EXPRESSIOON\images\train"
validation_data_dir = r"E:\ML\FACE EXPRESSIOON\images\validation"

train_data_gen = ImageDataGenerator(rescale = 1./255,
                                    rotation_range = 30,
                                    shear_range = 0.3,
                                    zoom_range = 3.0,
                                    width_shift_range = 0.4,
                                    height_shift_range = 0.4,
                                    horizontal_flip = True)

validation_data_gen = ImageDataGenerator(rescale = 1./255)

train_generator = train_data_gen.flow_from_directory(train_data_dir,
                                                    color_mode = "grayscale",
                                                    target_size = (img_cols,img_rows)
                                                    ,batch_size = batch_size,
                                                    class_mode = "categorical",
                                                    shuffle = True)

validation_generator = validation_data_gen.flow_from_directory(validation_data_dir,
                                                                color_mode = "grayscale",
                                                                target_size = (img_cols,img_rows),
                                                                batch_size = batch_size,
                                                                class_mode = "categorical",
                                                                shuffle = True)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [3]:
model = Sequential()

#block1
model.add(Conv2D(32,(3,3),padding="same",kernel_initializer ="he_normal",input_shape=(img_rows,img_cols,1)))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(Conv2D(32,(3,3),padding="same",kernel_initializer ="he_normal",input_shape=(img_rows,img_cols,1)))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [4]:
#block2
model.add(Conv2D(64,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(Conv2D(64,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [5]:
#block3
model.add(Conv2D(128,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(Conv2D(128,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [6]:
#block4
model.add(Conv2D(256,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(Conv2D(256,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

In [7]:
#block4
model.add(Conv2D(512,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(Conv2D(512,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [8]:
#block5
model.add(Flatten())
model.add(Dense(64,kernel_initializer="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [9]:
#block6
model.add(Dense(64,kernel_initializer="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [10]:
model.add(Dense(num_classes,kernel_initializer="he_normal"))
model.add(Activation("softmax"))

In [11]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)       

In [12]:
from keras.optimizers import RMSprop , SGD , Adam
from keras.callbacks import ModelCheckpoint , EarlyStopping , ReduceLROnPlateau

In [13]:
checkpoint = ModelCheckpoint("checkpoint2.h5",
                        monitor="val_loss",
                        mode="min",
                        save_best_only=True,
                        verbose=1)

earlystop = EarlyStopping(monitor="val_loss",
                        min_delta=0,
                        patience=5,
                        restore_best_weights=True,
                        verbose=1)

reduce_lr = ReduceLROnPlateau(monitor="val_loss",
                             factor=0.2,
                             patience=5,
                             verbose=1,
                             min_delta=0.0001)

In [ ]:
callbacks=[earlystop,checkpoint,reduce_lr]

model.compile(loss="categorical_crossentropy",
             optimizer=Adam(lr=0.001),
             metrics=["accuracy"] )

nb_train_samples = 28821
nb_validation_samples = 7066
epochs = 25

history = model.fit_generator(train_generator,
                             steps_per_epoch=nb_train_samples//batch_size,
                             epochs=epochs,
                             callbacks=callbacks,
                             validation_data=validation_generator,
                             validation_steps=nb_validation_samples//batch_size)

Epoch 1/25
28821/28821 [==============================] - 4833s 168ms/step - loss: 1.8189 - accuracy: 0.2467 - val_loss: 2397.0347 - val_accuracy: 0.1417

Epoch 00001: val_loss improved from inf to 2397.03467, saving model to checkpoint2.h5
Epoch 2/25
  681/28821 [..............................] - ETA: 1:15:16 - loss: 1.7998 - accuracy: 0.2614